In [5]:
!python -V

Python 3.9.12


In [53]:
import pandas as pd
import numpy as np
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [34]:
def variable_creation(datafr):
    return(
        datafr.assign(
            duration = datafr.lpep_dropoff_datetime - datafr.lpep_pickup_datetime
        )
    )


def data_type_correction(datafr, categorical):
    datafr.loc[:,categorical] = datafr.loc[:,categorical].astype(str)
    return(datafr)

def data_pull(path_to_file, categorical):
    return(
        (pd.read_parquet('../data/green_tripdata_2021-01.parquet')
        .pipe(variable_creation)
        .pipe(data_type_correction, categorical))
    )

## Data pull 

In [38]:
path_data_file = './data/green_tripdata_2021-01.parquet'

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df = data_pull(path_data_file, categorical)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   VendorID               76518 non-null  int64          
 1   lpep_pickup_datetime   76518 non-null  datetime64[ns] 
 2   lpep_dropoff_datetime  76518 non-null  datetime64[ns] 
 3   store_and_fwd_flag     40471 non-null  object         
 4   RatecodeID             40471 non-null  float64        
 5   PULocationID           76518 non-null  object         
 6   DOLocationID           76518 non-null  object         
 7   passenger_count        40471 non-null  float64        
 8   trip_distance          76518 non-null  float64        
 9   fare_amount            76518 non-null  float64        
 10  extra                  76518 non-null  float64        
 11  mta_tax                76518 non-null  float64        
 12  tip_amount             76518 non-null  float64

In [57]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values.astype('float64')

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

3528404074210.377

In [59]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [61]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [62]:
len(df_train), len(df_val)

(73908, 61921)

In [63]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [64]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [65]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [66]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479556169484162

In [69]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [70]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.167275941179728